# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-17-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-17-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-07-18 04:22:07,33.93911,67.709953,137853,5983,82586.0,49284.0,Afghanistan,354.119894,4.340130
1,NaN,NaN,NaN,Albania,2021-07-18 04:22:07,41.15330,20.168300,132665,2456,130072.0,137.0,Albania,4609.945097,1.851280
2,NaN,NaN,NaN,Algeria,2021-07-18 04:22:07,28.03390,1.659600,152210,3922,104992.0,43296.0,Algeria,347.106909,2.576703
3,NaN,NaN,NaN,Andorra,2021-07-18 04:22:07,42.50630,1.521800,14273,127,13844.0,302.0,Andorra,18472.788455,0.889792
4,NaN,NaN,NaN,Angola,2021-07-18 04:22:07,-11.20270,17.873900,40707,954,34732.0,5021.0,Angola,123.856472,2.343577


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,130113,131586,131586,131586,134653,134653,136643,137853,137853,137853
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132565,132580,132587,132592,132597,132608,132616,132629,132647,132665
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,143652,144483,145296,146064,146942,147883,148797,149906,151103,152210


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,5477,5561,5561,5561,5791,5791,5923,5983,5983,5983
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2456,2456,2456,2456,2456,2456,2456,2456,2456,2456
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3798,3811,3824,3836,3851,3867,3882,3895,3910,3922


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,78521,79446,80226,81077,82586,82586,82586,82586,82586,82586
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,130037,130043,130047,130050,130052,130053,130059,130061,130067,130072
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,99931,100383,100911,101430,102010,102594,103192,103809,104397,104992


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7277,7294,7299,7299,7306,7306,7320,7329,7341,7346
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,22154,22220,22267,22267,22337,22337,22464,22556,22642,22734
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2354,2361,2365,2365,2367,2367,2373,2377,2386,2389


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,316,317,318,318,318,318,321,323,323,323
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,60,60,60,60,60,60,60,60,60,60


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-07-18 04:22:07,32.539527,-86.644082,7346,113,NaN,NaN,"Autauga, Alabama, US",13148.615511,1.538252
693,1075.0,Lamar,Alabama,US,2021-07-18 04:22:07,33.779950,-88.096680,1483,38,NaN,NaN,"Lamar, Alabama, US",10742.484607,2.562374
694,1077.0,Lauderdale,Alabama,US,2021-07-18 04:22:07,34.901719,-87.656247,9793,253,NaN,NaN,"Lauderdale, Alabama, US",10560.881709,2.583478


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,34067669,608882,0.0
India,31064908,413091,30227792.0
Brazil,19342448,541266,17254329.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,34067669,608882,0.0,33458787.0,2021-07-18 04:22:07,37.950547,-91.419547
India,31064908,413091,30227792.0,424025.0,2021-07-18 04:22:07,23.088275,81.806127
Brazil,19342448,541266,17254329.0,1546853.0,2021-07-18 04:22:07,-12.669522,-48.480493
France,5917397,111657,408709.0,5397031.0,2021-07-18 04:22:07,6.430808,-34.730285
Russia,5860113,145222,5258917.0,455974.0,2021-07-18 04:22:07,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3858148,64091,0.0
Texas,3039664,52816,0.0
Florida,2450344,38388,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3858148,64091,0.0,3794057.0,2021-07-18 04:22:07,37.843962,-120.728594
Texas,3039664,52816,0.0,2986848.0,2021-07-18 04:22:07,31.660643,-98.653069
Florida,2450344,38388,0.0,2411956.0,2021-07-18 04:22:07,28.940755,-82.700744
New York,2125535,53765,0.0,2071770.0,2021-07-18 04:22:07,42.544151,-75.474183
Illinois,1399946,25790,0.0,1374156.0,2021-07-18 04:22:07,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1266227,24582,0.0
Arizona,Maricopa,567576,10381,0.0
Illinois,Cook,558928,10539,0.0
Florida,Miami-Dade,522734,6472,0.0
Texas,Harris,407616,6659,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1266227,24582,0.0,1241645.0,2021-07-18 04:22:07,34.308284,-118.228241,6037.0
Arizona,Maricopa,567576,10381,0.0,557195.0,2021-07-18 04:22:07,33.348359,-112.491815,4013.0
Illinois,Cook,558928,10539,0.0,548389.0,2021-07-18 04:22:07,41.841448,-87.816588,17031.0
Florida,Miami-Dade,522734,6472,0.0,516262.0,2021-07-18 04:22:07,25.611236,-80.551706,12086.0
Texas,Harris,407616,6659,0.0,400957.0,2021-07-18 04:22:07,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-15,137853,132629,149906,14239,40530,1267,4719952,226949,31626,652660,...,5301296,378041,117902,4,289362,42288,315504,6967,182129,78872
2021-07-16,137853,132647,151103,14273,40631,1268,4737213,227111,31771,653001,...,5353245,378480,118380,4,290524,46292,315504,6972,183344,81168
2021-07-17,137853,132665,152210,14273,40707,1268,4749443,227298,31899,653412,...,5407428,378733,118881,4,291716,51002,315641,6977,184672,82613


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-15,5983,2456,3895,127,951,42,100695,4556,912,10727,...,128864,5865,785,1,3339,207,3585,1366,2991,2418
2021-07-16,5983,2456,3910,127,952,42,101158,4558,913,10728,...,128913,5871,789,1,3350,225,3585,1366,3022,2520
2021-07-17,5983,2456,3922,127,954,42,101434,4559,914,10728,...,128960,5879,793,1,3362,225,3587,1368,3052,2588


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-15,82586,130061,103809,13829,34620,1222,4345549,218022,23920,639239,...,16133,366690,112941,3,272029,9688,310329,4154,166388,49774
2021-07-16,82586,130067,104397,13844,34724,1224,4363105,218128,23935,639352,...,16220,367457,113261,3,273341,10020,310329,4154,168052,51139
2021-07-17,82586,130072,104992,13844,34732,1224,4379169,218242,23939,639422,...,16429,367967,113825,3,274522,10312,310525,4156,169320,51923


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7277,7294,7299,7299,7306,7306,7320,7329,7341,7346
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1478,1478,1478,1478,1479,1479,1481,1482,1482,1483
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9674,9698,9703,9703,9720,9720,9754,9767,9782,9793
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3146,3147,3150,3150,3150,3150,3155,3160,3161,3162
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16390,16422,16448,16448,16454,16454,16483,16513,16541,16561


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-15,557578,72153,0,904865,361238,3849540,564686,350536,110231,49,...,872934,3032699,421950,24529,4119,684054,459306,164843,679604,63455
2021-07-16,558577,72440,0,906116,362580,3855167,565294,350675,110270,49,...,872945,3037130,422619,24550,4160,684499,460067,164933,679826,63523
2021-07-17,559478,72440,0,906116,362580,3858148,565829,350675,110339,49,...,872945,3039664,422619,24550,4160,684499,460067,164933,679826,63523


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-07-15        7329   22556    2377  2713   7059    1251   2289   14863   
2021-07-16        7341   22642    2386  2719   7073    1251   2296   14879   
2021-07-17        7346   22734    2389  2721   7090    1252   2300   14902   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-15         3776     1899  ...    2922    716     3282      817   
2021-07-16         3781     1903  ...    2926    716     3283      819   
2021-07-17         3784     1908  ...    2926    716     3283      819   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-15           4876  3834  2333          0      932    664  
2021-07-16           4882  3835  2335          0      933    664  
2021-07-17           4882  3835  2335          0      933    664  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-15,11439,381,0,18083,5981,64043,6872,8282,1696,0,...,12628,52763,2410,258,32,11471,6036,2917,8198,760
2021-07-16,11442,381,0,18100,5992,64086,6876,8282,1697,0,...,12533,52792,2413,258,32,11477,6042,2919,8209,760
2021-07-17,11443,381,0,18100,5992,64091,6886,8282,1697,0,...,12533,52816,2413,258,32,11477,6042,2919,8209,760


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-15         113     323      60   65    139      42     71     331   
2021-07-16         113     323      60   65    139      42     71     331   
2021-07-17         113     323      60   65    139      42     71     331   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-15          124       47  ...      33     14       31        7   
2021-07-16          124       47  ...      33     14       31        7   
2021-07-17          124       47  ...      33     14       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-15             41    11    13          0       26      6  
2021-07-16             41    11    13          0       26      6  
2021-07-17             41    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-15,0.008855,0.000098,0.007453,0.005082,0.005034,0.000000,0.003678,0.000851,0.003586,0.000469,...,0.009260,0.000892,0.004199,0.0,0.004384,0.105887,0.000301,0.000431,0.008751,0.032613
2021-07-16,0.000000,0.000136,0.007985,0.002388,0.002492,0.000789,0.003657,0.000714,0.004585,0.000522,...,0.009799,0.001161,0.004054,0.0,0.004016,0.094684,0.000000,0.000718,0.006671,0.029110
2021-07-17,0.000000,0.000136,0.007326,0.000000,0.001870,0.000000,0.002582,0.000823,0.004029,0.000629,...,0.010122,0.000668,0.004232,0.0,0.004103,0.101745,0.000434,0.000717,0.007243,0.017803


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-15,0.01013,0.0,0.003349,0.0,0.005285,0.0,0.004439,0.000879,0.000000,0.000093,...,0.000520,0.001879,0.005122,0.0,0.003607,0.500000,0.000000,0.000000,0.014242,0.036878
2021-07-16,0.00000,0.0,0.003851,0.0,0.001052,0.0,0.004598,0.000439,0.001096,0.000093,...,0.000380,0.001023,0.005096,0.0,0.003294,0.086957,0.000000,0.000000,0.010364,0.042184
2021-07-17,0.00000,0.0,0.003069,0.0,0.002101,0.0,0.002728,0.000219,0.001095,0.000000,...,0.000365,0.001363,0.005070,0.0,0.003582,0.000000,0.000558,0.001464,0.009927,0.026984


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-15,0.0,0.000015,0.005979,0.002392,0.001041,0.000000,0.004425,0.000578,0.000376,0.000133,...,0.000372,0.002291,0.003510,0.0,0.003812,0.006650,0.000290,0.000723,0.008877,0.014698
2021-07-16,0.0,0.000046,0.005664,0.001085,0.003004,0.001637,0.004040,0.000486,0.000627,0.000177,...,0.005393,0.002092,0.002833,0.0,0.004823,0.034269,0.000000,0.000000,0.010001,0.027424
2021-07-17,0.0,0.000038,0.005699,0.000000,0.000230,0.000000,0.003682,0.000523,0.000167,0.000109,...,0.012885,0.001388,0.004980,0.0,0.004321,0.029142,0.000632,0.000481,0.007545,0.015331


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-15,0.001734,0.000000,NaN,0.001122,0.002720,0.000466,0.000925,0.000428,0.000445,0.0,...,0.000656,0.000998,0.001645,0.000530,0.018798,0.000644,0.001721,0.000486,0.000316,0.001958
2021-07-16,0.001792,0.003978,NaN,0.001383,0.003715,0.001462,0.001077,0.000397,0.000354,0.0,...,0.000013,0.001461,0.001585,0.000856,0.009954,0.000651,0.001657,0.000546,0.000327,0.001072
2021-07-17,0.001613,0.000000,NaN,0.000000,0.000000,0.000773,0.000946,0.000000,0.000626,0.0,...,0.000000,0.000834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-07-15      0.001230  0.004095  0.001686  0.001846  0.000567  0.000800   
2021-07-16      0.001637  0.003813  0.003786  0.002212  0.001983  0.000000   
2021-07-17      0.000681  0.004063  0.001257  0.000736  0.002404  0.000799   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-15      0.000437  0.000135  0.000795  0.000527  ...  0.001714   
2021-07-16      0.003058  0.001076  0.001324  0.002106  ...  0.001369   
2021-07-17      0.001742  0.001546  0.000793  0.002627  ...  0.000000   

Province_State                                                              \
Admin2           Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-15      0.01273  0.000305  0.006158   0.001438  0.000783  0.000429   
2021-07-16      0.00000  0.000305  0.002448   0.001231  0.000261  0.000857   
2021-07-17      0.00000  0.000000  0.000000   0.000000  0.000000  0.000000   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2021-07-15            NaN  0.003229    0.0  
2021-07-16            NaN  0.001073    0.0  
2021-07-17            NaN  0.000000    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-15,0.000963,0.0,NaN,0.000387,0.000669,0.000484,0.000437,0.000362,0.00000,NaN,...,0.000238,0.000683,0.001663,0.0,0.0,0.000349,0.001493,0.000343,0.000244,0.0
2021-07-16,0.000262,0.0,NaN,0.000940,0.001839,0.000671,0.000582,0.000000,0.00059,NaN,...,-0.007523,0.000550,0.001245,0.0,0.0,0.000523,0.000994,0.000686,0.001342,0.0
2021-07-17,0.000087,0.0,NaN,0.000000,0.000000,0.000078,0.001454,0.000000,0.00000,NaN,...,0.000000,0.000455,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-15         0.0  0.006231     0.0  0.0    0.0     0.0    0.0     0.0   
2021-07-16         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2021-07-17         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-15          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-07-16          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-07-17          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-15            0.0   0.0   0.0        NaN      0.0    0.0  
2021-07-16            0.0   0.0   0.0        NaN      0.0    0.0  
2021-07-17            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-15,0.00976,0.000080,0.006778,0.003034,0.004259,0.000201,0.003785,0.000731,0.003280,0.000437,...,0.008353,0.000971,0.004191,4.308232e-27,0.004210,0.092198,0.000324,0.000562,0.009222,0.035886
2021-07-16,0.00488,0.000108,0.007381,0.002711,0.003376,0.000495,0.003721,0.000723,0.003932,0.000480,...,0.009076,0.001066,0.004122,2.154116e-27,0.004113,0.093441,0.000162,0.000640,0.007946,0.032498
2021-07-17,0.00244,0.000122,0.007354,0.001355,0.002623,0.000248,0.003151,0.000773,0.003981,0.000555,...,0.009599,0.000867,0.004177,1.077058e-27,0.004108,0.097593,0.000298,0.000678,0.007595,0.025150


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-15,0.013580,4.129533e-10,0.003609,4.377056e-25,0.004768,2.433295e-19,0.004737,0.000734,0.000086,0.000109,...,0.000423,0.001828,0.004708,0.0,0.003788,0.274487,0.000237,0.000011,0.014717,0.029966
2021-07-16,0.006790,2.064766e-10,0.003730,2.188528e-25,0.002910,1.216647e-19,0.004667,0.000586,0.000591,0.000101,...,0.000402,0.001426,0.004902,0.0,0.003541,0.180722,0.000119,0.000006,0.012541,0.036075
2021-07-17,0.003395,1.032383e-10,0.003400,1.094264e-25,0.002505,6.083237e-20,0.003698,0.000403,0.000843,0.000050,...,0.000383,0.001394,0.004986,0.0,0.003562,0.090361,0.000338,0.000735,0.011234,0.031529


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-15,0.001853,0.000023,0.005830,0.001364,0.001824,2.500466e-08,0.003829,0.000478,0.000344,0.000142,...,0.002567,0.002172,0.003905,1.033976e-25,0.004131,0.009516,0.000379,0.000865,0.009313,0.018453
2021-07-16,0.000927,0.000034,0.005747,0.001224,0.002414,8.183431e-04,0.003935,0.000482,0.000486,0.000160,...,0.003980,0.002132,0.003369,5.169879e-26,0.004477,0.021893,0.000190,0.000432,0.009657,0.022938
2021-07-17,0.000463,0.000036,0.005723,0.000612,0.001322,4.091716e-04,0.003808,0.000502,0.000326,0.000135,...,0.008433,0.001760,0.004174,2.584939e-26,0.004399,0.025517,0.000411,0.000457,0.008601,0.019135


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-15,0.001602,0.001034,NaN,0.001211,0.003189,0.000668,0.000907,0.000406,0.000439,1.703901e-147,...,0.000741,0.001170,0.001510,0.000587,0.010253,0.000576,0.001784,0.000474,0.000314,0.001748
2021-07-16,0.001697,0.002506,NaN,0.001297,0.003452,0.001065,0.000992,0.000401,0.000397,8.519503e-148,...,0.000377,0.001315,0.001548,0.000722,0.010103,0.000613,0.001721,0.000510,0.000320,0.001410
2021-07-17,0.001655,0.001253,NaN,0.000648,0.001726,0.000919,0.000969,0.000201,0.000511,4.259751e-148,...,0.000188,0.001075,0.000774,0.000361,0.005052,0.000307,0.000860,0.000255,0.000160,0.000705


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-07-15      0.001187  0.003732  0.001585  0.001361  0.001393  0.0004   
2021-07-16      0.001412  0.003772  0.002686  0.001786  0.001688  0.0002   
2021-07-17      0.001047  0.003918  0.001971  0.001261  0.002046  0.0005   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-15      0.000933  0.000468  0.001144  0.000584  ...  0.001464   
2021-07-16      0.001995  0.000772  0.001234  0.001345  ...  0.001416   
2021-07-17      0.001869  0.001159  0.001014  0.001986  ...  0.000708   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-15      0.007701  0.000214  0.004671   0.001684  0.000936  0.000491   
2021-07-16      0.003851  0.000259  0.003559   0.001457  0.000599  0.000674   
2021-07-17      0.001925  0.000130  0.001780   0.000729  0.000299  0.000337   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-07-15           -1.0  0.001623  0.000018  
2021-07-16           -1.0  0.001348  0.000009  
2021-07-17           -1.0  0.000674  0.000005  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-15,0.001069,0.000371,NaN,0.000658,0.001033,0.000449,0.000719,0.000181,0.000148,NaN,...,0.000443,0.000642,0.001582,3.567002e-07,0.000130,0.000457,0.001474,0.000789,0.000448,0.001503
2021-07-16,0.000666,0.000186,NaN,0.000799,0.001436,0.000560,0.000651,0.000091,0.000369,NaN,...,-0.003540,0.000596,0.001413,1.783501e-07,0.000065,0.000490,0.001234,0.000737,0.000895,0.000752
2021-07-17,0.000377,0.000093,NaN,0.000400,0.000718,0.000319,0.001053,0.000045,0.000184,NaN,...,-0.001770,0.000525,0.000707,8.917504e-08,0.000033,0.000245,0.000617,0.000369,0.000447,0.000376


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                                  \
Admin2               Autauga   Baldwin       Barbour      Bibb        Blount   
2021-07-15      1.854809e-14  0.005554  2.020496e-09  0.000031  1.590599e-21   
2021-07-16      9.274043e-15  0.002777  1.010248e-09  0.000015  7.952994e-22   
2021-07-17      4.637022e-15  0.001389  5.051241e-10  0.000008  3.976497e-22   

Province_State                                                            ...  \
Admin2               Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2021-07-15      5.289122e-21  1.309052e-17  0.000006  0.002032  0.000347  ...   
2021-07-16      2.644561e-21  6.545259e-18  0.000003  0.001016  0.000174  ...   
2021-07-17      1.322280e-21  3.272629e-18  0.000001  0.000508  0.000087  ...   

Province_State   Wyoming                                                   \
Admin2              Park    Platte      Sheridan      Sublette Sweetwater   
2021-07-15      0.000031  0.009616  6.270579e-39  2.754722e-59   0.003125   
2021-07-16      0.000016  0.004808  3.135289e-39  1.377361e-59   0.001563   
2021-07-17      0.000008  0.002404  1.567645e-39  6.886804e-60   0.000781   

Province_State                                                       \
Admin2                 Teton         Uinta Unassigned      Washakie   
2021-07-15      1.616879e-12  8.823260e-24   0.752941  1.711555e-51   
2021-07-16      8.084397e-13  4.411630e-24   0.752941  8.557774e-52   
2021-07-17      4.042199e-13  2.205815e-24   0.752941  4.278887e-52   

Province_State                
Admin2                Weston  
2021-07-15      1.455192e-12  
2021-07-16      7.275958e-13  
2021-07-17      3.637979e-13  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
